
### 🔄 Trigger Mode Cheat Sheet

**Always‑on, low‑latency ingestion**  
`ProcessingTime("15s" – "5m")`

**Single-run or nightly scheduled job**  
`Once`

**Backfill / catch‑up until input empty**  
`AvailableNow`

**Ultra‑low latency (experimental)**  
`Continuous`



In [0]:
%sql
drop table datacentre.bronze.order_autoloader1

In [0]:
schema_location = "/Volumes/datacentre/bronze/autoloader1/schema/"
input_location = "/Volumes/datacentre/bronze/autoloader1/input/"
checkpoint_location = "/Volumes/datacentre/bronze/autoloader1/checkpoints/"

(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "csv")
 .option("cloudFiles.inferSchema", "true")
 .option("cloudFiles.inferColumnTypes", "true")
 .option("cloudFiles.schemaEvolutionMode", "rescue")
 .option("cloudFiles.schemaLocation", f"{schema_location}")
 .load(f"{input_location}")
 .writeStream
 .option("checkpointLocation", f"{checkpoint_location}")
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .table("datacentre.bronze.order_autoloader1")
)

In [0]:
# List files in the input directory
import os
input_dir = "/Volumes/datacentre/bronze/autoloader/input/"
files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

if files:
    sample_file = os.path.join(input_dir, files[0])
    df = spark.read.csv(sample_file, header=True, inferSchema=True)
    print(f"Schema for {sample_file}:")
    df.printSchema()
    print("Sample data:")
    display(df.limit(5))
else:
    print("No CSV files found in input directory.")

In [0]:
spark.sql("select * from datacentre.bronze.order_autoloader1").display()